# LangChain Agent with Qiskit Code Assistant MCP Server

This notebook demonstrates how to create an AI agent using LangChain that connects to the **qiskit-code-assistant-mcp-server** via the Model Context Protocol (MCP).

The agent can interact with Qiskit Code Assistant to:
- Generate quantum code completions
- Answer questions about Qiskit and quantum computing (RAG mode)
- Accept model disclaimers
- Track completion acceptance

## Architecture

```
┌─────────────┐     MCP Protocol     ┌──────────────────────────────────┐
│  LangChain  │ ◄──────────────────► │ qiskit-code-assistant-mcp-server │
│    Agent    │                      │                                  │
└─────────────┘                      │  ┌────────────────────────────┐  │
                                     │  │  Qiskit Code Assistant API │  │
                                     │  └────────────────────────────┘  │
                                     │               │                  │
                                     └───────────────│──────────────────┘
                                                     ▼
                                            ┌─────────────────┐
                                            │  IBM Quantum    │
                                            │  Code Assistant │
                                            └─────────────────┘
```

## Setup

### 1. Install Dependencies

Run these commands in your terminal:

```bash
# Install the MCP server with examples dependencies
pip install qiskit-code-assistant-mcp-server[examples]

# Install your preferred LLM provider (choose one):
pip install langchain-openai       # For OpenAI
pip install langchain-anthropic    # For Anthropic Claude
pip install langchain-google-genai # For Google Gemini
pip install langchain-ollama       # For local Ollama
pip install langchain-ibm          # For IBM Watsonx
```

### 2. Configure Environment Variables

Set your IBM Quantum token for the Qiskit Code Assistant.

You can either:
- Set them in a `.env` file in this directory
- Set them as environment variables
- Enter them in the cell below

In [1]:
import os

from dotenv import load_dotenv

# LangChain imports
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_mcp_adapters.tools import load_mcp_tools


# Load from .env file if it exists
load_dotenv()

# Or set directly (uncomment and fill in):
# os.environ["QISKIT_IBM_TOKEN"] = "your-ibm-quantum-token"

# Set your LLM provider API key (uncomment the one you're using):
# os.environ["OPENAI_API_KEY"] = "your-openai-api-key"
# os.environ["ANTHROPIC_API_KEY"] = "your-anthropic-api-key"
# os.environ["GOOGLE_API_KEY"] = "your-google-api-key"

# Verify configuration
print("Configuration status:")
print(f"  QISKIT_IBM_TOKEN: {'✓ Set' if os.getenv('QISKIT_IBM_TOKEN') else '✗ Not set'}")

Configuration status:
  QISKIT_IBM_TOKEN: ✓ Set


## Choose Your LLM Provider

Run **one** of the following cells based on your preferred LLM provider:

In [ ]:
# Option 1: OpenAI
from langchain_openai import ChatOpenAI


llm = ChatOpenAI(model="gpt-4o", temperature=0)
print("Using OpenAI GPT-4o")

In [ ]:
# Option 2: Anthropic Claude
from langchain_anthropic import ChatAnthropic


llm = ChatAnthropic(model="claude-sonnet-4-20250514", temperature=0)
print("Using Anthropic Claude Sonnet")

Using Anthropic Claude Sonnet


In [ ]:
# Option 3: Google Gemini
from langchain_google_genai import ChatGoogleGenerativeAI


llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro", temperature=0)
print("Using Google Gemini Pro")

In [ ]:
# Option 4: Local Ollama (no API key needed)
from langchain_ollama import ChatOllama


llm = ChatOllama(model="llama3.2", temperature=0)
print("Using local Ollama with Llama 3.2")

In [ ]:
# Option 5: IBM Watsonx
from langchain_ibm import ChatWatsonx


llm = ChatWatsonx(
    model_id="ibm/granite-3-8b-instruct",
    url=os.getenv("WATSONX_URL", "https://us-south.ml.cloud.ibm.com"),
    project_id=os.getenv("WATSONX_PROJECT_ID"),
    params={"temperature": 0, "max_tokens": 4096},
)
print("Using IBM Watsonx Granite")

## Define the System Prompt

This prompt tells the agent what it can do and how to behave:

In [3]:
SYSTEM_PROMPT = """You are a helpful quantum computing coding assistant with access to Qiskit Code Assistant
through the MCP server.

You can help users:
- Generate quantum code using Qiskit (qca_get_completion tool)
- Answer questions about Qiskit and quantum computing concepts (qca_get_rag_completion tool)
- Accept model disclaimers when required (qca_accept_model_disclaimer tool)
- Track completion acceptance for feedback (qca_accept_completion tool)

When generating code:
- Use the qca_get_completion tool to generate Qiskit code based on user prompts
- Provide clear explanations of the generated code
- Suggest improvements or alternatives when appropriate

When answering questions:
- Use the qca_get_rag_completion tool for conceptual questions about quantum computing
- Provide accurate and educational explanations
- Reference Qiskit documentation when relevant

If you encounter a disclaimer requirement, use qca_accept_model_disclaimer to accept it.
After using a completion, use qca_accept_completion to provide feedback."""

## Create the MCP Client

This configures the connection to the qiskit-code-assistant-mcp-server:

In [4]:
def get_mcp_client():
    """Create and return an MCP client configured for the Qiskit Code Assistant server."""
    return MultiServerMCPClient(
        {
            "qiskit-code-assistant": {
                "transport": "stdio",
                "command": "qiskit-code-assistant-mcp-server",
                "args": [],
                "env": {
                    "QISKIT_IBM_TOKEN": os.getenv("QISKIT_IBM_TOKEN", ""),
                },
            }
        }
    )

## Create the Agent

Now we'll create a function that sets up the agent with a persistent MCP session.

Using a persistent session is important because it:
- Keeps a single MCP server process running
- Reuses the Qiskit Code Assistant connection
- Makes tool calls much faster

In [5]:
async def create_agent_with_session(session):
    """Create a LangChain agent using an existing MCP session."""
    # Load tools from the existing session
    tools = await load_mcp_tools(session)
    print(f"Loaded {len(tools)} tools from MCP server:")
    for tool in tools:
        print(f"  - {tool.name}")

    # Create the agent using LangChain's create_agent
    agent = create_agent(llm, tools, system_prompt=SYSTEM_PROMPT)
    return agent

## Helper Function to Run Queries

This function sends a query to the agent and returns the response:

In [6]:
async def ask_agent(agent, query: str) -> str:
    """Send a query to the agent and return the response."""
    result = await agent.ainvoke({"messages": [HumanMessage(content=query)]})
    messages = result.get("messages", [])
    if messages:
        return messages[-1].content
    return "No response generated."

## Run the Agent

Now let's create the agent and ask it some questions!

The following cell starts the MCP server, creates the agent, and keeps the session open for multiple queries:

In [7]:
# Create MCP client and start a persistent session
mcp_client = get_mcp_client()

print("Starting MCP server and creating agent...")
print("(This may take a few seconds on first run)\n")

Starting MCP server and creating agent...
(This may take a few seconds on first run)



### Example 1: Generate a Bell State Circuit

Let's ask the agent to generate code for a Bell state:

In [8]:
async with mcp_client.session("qiskit-code-assistant") as session:
    agent = await create_agent_with_session(session)

    response = await ask_agent(agent, "Write a quantum circuit that creates a Bell state")
    print(response)

Loaded 4 tools from MCP server:
  - qca_accept_model_disclaimer
  - qca_get_completion
  - qca_get_rag_completion
  - qca_accept_completion
[]


### Example 2: Ask a Quantum Computing Question

Use RAG mode to get answers about quantum computing concepts:

In [9]:
async with mcp_client.session("qiskit-code-assistant") as session:
    agent = await create_agent_with_session(session)

    response = await ask_agent(agent, "What is quantum entanglement and why is it important?")
    print(response)

Loaded 4 tools from MCP server:
  - qca_accept_model_disclaimer
  - qca_get_completion
  - qca_get_rag_completion
  - qca_accept_completion
## What is Quantum Entanglement?

**Quantum entanglement** is a phenomenon where two or more qubits become interconnected such that the state of one qubit is dependent on the state of the other, regardless of the distance between them. This means that the measurement of one qubit instantly affects the state of the other, even if they are light-years apart. This property is fundamental to quantum computing and is one of the core concepts that differentiates it from classical computing.

## Why is Quantum Entanglement Important?

1. **Superposition and Parallelism**: Entanglement allows qubits to exist in multiple states simultaneously, enabling quantum computers to process a vast number of possibilities at once. This parallelism is crucial for solving complex problems more efficiently than classical computers.

2. **Quantum Communication**: Entangle

### Example 3: Generate a VQE Circuit

Let's generate more complex quantum code:

In [10]:
async with mcp_client.session("qiskit-code-assistant") as session:
    agent = await create_agent_with_session(session)

    response = await ask_agent(
        agent, "Generate code to set up a simple VQE algorithm for finding the ground state energy"
    )
    print(response)

Loaded 4 tools from MCP server:
  - qca_accept_model_disclaimer
  - qca_get_completion
  - qca_get_rag_completion
  - qca_accept_completion
Perfect! I've generated a complete VQE setup for finding the ground state energy. Let me explain the code:

## Overview of the VQE Algorithm Setup

**Key Components:**

1. **Molecule Definition**: The code uses a hydrogen molecule (H2) as an example with atoms separated by 0.735 Ångströms.

2. **Driver Setup**: Uses `PySCFDriver` to compute the electronic structure problem with:
   - STO-3G basis set (minimal basis)
   - Neutral charge and singlet spin state

3. **Hamiltonian Mapping**: 
   - Extracts the fermionic Hamiltonian from the problem
   - Maps it to qubit operators using the `ParityMapper` (reduces qubit count)

4. **Ansatz (Variational Form)**: 
   - Uses `TwoLocal` circuit with:
     - RY and RZ rotation blocks
     - CZ entanglement gates
     - Full entanglement pattern
     - 3 repetitions for expressibility

5. **Estimator**: Uses Q

### Example 4: Explain Qiskit Concepts

Ask about Qiskit-specific features:

In [11]:
async with mcp_client.session("qiskit-code-assistant") as session:
    agent = await create_agent_with_session(session)

    response = await ask_agent(agent, "How does Qiskit's transpiler work?")
    print(response)

Loaded 4 tools from MCP server:
  - qca_accept_model_disclaimer
  - qca_get_completion
  - qca_get_rag_completion
  - qca_accept_completion
Great! Here's a comprehensive explanation of how Qiskit's transpiler works:

## **Overview**

Qiskit's transpiler transforms abstract quantum circuits into forms suitable for execution on specific quantum devices. It's a multi-stage process that handles everything from gate decomposition to qubit mapping and optimization.

## **Key Transpiler Stages**

1. **Init Stage**: Prepares the circuit by translating multi-qubit gates into single- and two-qubit gates.

2. **Layout Stage**: Maps virtual qubits to physical qubits on the device using methods like:
   - TrivialLayout (naive mapping)
   - VF2Layout (subgraph isomorphism)
   - DenseLayout (most connected subgraph)
   - SabreLayout (heuristic-based)

3. **Routing Stage**: Inserts SWAP gates to match the circuit's connectivity requirements to the device's physical connectivity.

4. **Translation Stag

### Example 5: Interactive Chat

Run this cell to have an interactive conversation with the agent:

In [12]:
async with mcp_client.session("qiskit-code-assistant") as session:
    agent = await create_agent_with_session(session)
    print("Agent ready! Type your questions below.")
    print("Enter 'quit' to stop.\n")

    while True:
        try:
            query = input("You: ").strip()
            if not query:
                continue
            if query.lower() in ["quit", "exit", "q"]:
                print("Goodbye!")
                break

            response = await ask_agent(agent, query)
            print(f"\nAssistant: {response}\n")
        except KeyboardInterrupt:
            print("\nGoodbye!")
            break

Loaded 4 tools from MCP server:
  - qca_accept_model_disclaimer
  - qca_get_completion
  - qca_get_rag_completion
  - qca_accept_completion
Agent ready! Type your questions below.
Enter 'quit' to stop.


Assistant: Hello! 👋 I'm your Qiskit Code Assistant, and I'm here to help you with quantum computing and Qiskit programming!

I can assist you with:

1. **Generating Quantum Code** - Write, complete, and optimize Qiskit code for quantum circuits and algorithms
2. **Answering Questions** - Explain quantum computing concepts and Qiskit features
3. **Code Optimization** - Help improve your existing quantum code

What would you like to do today? For example, you could:
- Ask me to write a quantum circuit for a specific task
- Ask conceptual questions about quantum computing
- Get help debugging or optimizing quantum code
- Learn about Qiskit features and best practices

Feel free to ask anything related to quantum computing or Qiskit! 🚀


Assistant: Great! Here's a comprehensive guide on ho

## Custom Queries

Use this cell to ask the agent any question:

In [13]:
# Enter your question here:
MY_QUESTION = "Write a circuit with 3 qubits that applies a Toffoli gate"

async with mcp_client.session("qiskit-code-assistant") as session:
    agent = await create_agent_with_session(session)
    response = await ask_agent(agent, MY_QUESTION)
    print(response)

Loaded 4 tools from MCP server:
  - qca_accept_model_disclaimer
  - qca_get_completion
  - qca_get_rag_completion
  - qca_accept_completion
[]


## Available Tools

The agent has access to these tools provided by the MCP server:

| Tool | Description |
|------|-------------|
| `qca_get_completion` | Generate Qiskit code completions based on prompts |
| `qca_get_rag_completion` | Answer questions about Qiskit and quantum computing using RAG |
| `qca_accept_model_disclaimer` | Accept the disclaimer for a model |
| `qca_accept_completion` | Accept/acknowledge a generated completion |

## Troubleshooting

### Authentication errors?
- Verify your `QISKIT_IBM_TOKEN` is correct
- Make sure you have access to Qiskit Code Assistant
- Get your token from: https://quantum.ibm.com/

### MCP server not found?
Make sure `qiskit-code-assistant-mcp-server` is installed: `pip install qiskit-code-assistant-mcp-server[examples]`

### Model disclaimer required?
The agent will automatically use `qca_accept_model_disclaimer` when needed.